# Create list of bounding boxes for images

## Initial Setup

Auto update from code base

In [1]:
%load_ext autoreload
%autoreload 2

Import libraries

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from tqdm import tqdm
import os
import json
import torch
from torchvision import transforms

from src.data import pil_loader

Load data

In [3]:
# Original CSVs
df_beauty_train = pd.read_csv('data/raw/beauty_data_info_train_competition.csv')
df_beauty_val = pd.read_csv('data/raw/beauty_data_info_val_competition.csv')

df_fashion_train = pd.read_csv('data/raw/fashion_data_info_train_competition.csv')
df_fashion_val = pd.read_csv('data/raw/fashion_data_info_val_competition.csv')

df_mobile_train = pd.read_csv('data/raw/mobile_data_info_train_competition.csv')
df_mobile_val = pd.read_csv('data/raw/mobile_data_info_val_competition.csv')
    
# Load label map
with open('data/derived/label_map_beauty.json', 'r') as file:
    label_map_beauty = json.load(file)
    
with open('data/derived/label_map_fashion.json', 'r') as file:
    label_map_fashion = json.load(file)
    
with open('data/derived/label_map_mobile.json', 'r') as file:
    label_map_mobile = json.load(file)

In [4]:
# Reverse label maps
rev_label_map_beauty = {v: k for k, v in label_map_beauty.items()}
rev_label_map_fashion = {v: k for k, v in label_map_fashion.items()}
rev_label_map_mobile = {v: k for k, v in label_map_mobile.items()}

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Inference for bounding boxes

In [6]:
transform_pipe = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
    ])

Beauty

In [7]:
# Load model checkpoint
checkpoint = 'models/BEST_checkpoint_beauty.pth.tar'
checkpoint = torch.load(checkpoint)
start_epoch = checkpoint['epoch'] + 1
best_loss = checkpoint['best_loss']
print('\nLoaded checkpoint from epoch %d. Best loss so far is %.3f.\n' % (start_epoch, best_loss))
model_beauty = checkpoint['model']
model_beauty = model_beauty.to(device)
model_beauty.eval()


Loaded checkpoint from epoch 55. Best loss so far is 3.173.



SSD300(
  (base): VGGBase(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), 

In [8]:
df_bb_beauty_train = pd.DataFrame()

with tqdm(total=len(df_beauty_train)) as pbar:
    for image_path in df_beauty_train['image_path']:
        image_path_full = os.path.join('data/raw', image_path)
        original_image = pil_loader(image_path_full)
        input_tensor = transform_pipe(original_image).unsqueeze(0).to(device)
        predicted_locs, predicted_scores = model_beauty(input_tensor)
        det_boxes, det_labels, det_scores = model_beauty.detect_objects(predicted_locs, predicted_scores, min_score=0.8,
                                                                        max_overlap=0.2, top_k=200)

        # Decode class integer labels
        det_labels = [rev_label_map_beauty[l] for l in det_labels[0].to('cpu').tolist()]
        det_labels_np = np.array(det_labels)

        # If no objects found, the bounding box coordinates would be the extremities of the original image
        if det_labels == ['background']:
            row_dict = {'image_path': image_path, 'x0': 0, 'y0':0,
                        'x1': original_image.size[0],'y1':original_image.size[1]}
            if len(df_bb_beauty_train) == 0:
                df_bb_beauty_train = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_beauty_train = df_bb_beauty_train.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Move detections to the CPU
        det_boxes = det_boxes[0].to('cpu')

        # Transform to original image dimensions and to NumPy array
        original_dims = torch.FloatTensor(
            [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
        det_boxes = det_boxes * original_dims
        det_boxes_np = det_boxes.detach().numpy().round().astype(int) + 1

        # Transform scores to NumPy array
        det_scores_np = det_scores[0].to('cpu').detach().numpy()

        # Prioritize 'beauty' label if it exists
        if 'beauty' in det_labels:
            det_scores_filtered = det_scores_np[det_labels_np == 'beauty']
            det_boxes_filtered = det_boxes_np[det_labels_np == 'beauty']

            det_boxes_best = det_boxes_filtered[det_scores_filtered.argmax()].flatten()

            row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
            if len(df_bb_beauty_train) == 0:
                df_bb_beauty_train = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_beauty_train = df_bb_beauty_train.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Record score
        det_boxes_best = det_boxes_np[det_scores_np.argmax()].flatten()
        row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
        if len(df_bb_beauty_train) == 0:
            df_bb_beauty_train = pd.DataFrame(row_dict,index=[0])
        else:
            df_bb_beauty_train = df_bb_beauty_train.append(row_dict, ignore_index=True)

        pbar.update()
        
df_bb_beauty_train.to_csv('data/derived/bb_beauty_train.csv', index=False)

100%|██████████| 286583/286583 [9:46:54<00:00,  8.03it/s]   


In [9]:
df_bb_beauty_val = pd.DataFrame()

with tqdm(total=len(df_beauty_val)) as pbar:
    for image_path in df_beauty_val['image_path']:
        image_path_full = os.path.join('data/raw', image_path)
        original_image = pil_loader(image_path_full)
        input_tensor = transform_pipe(original_image).unsqueeze(0).to(device)
        predicted_locs, predicted_scores = model_beauty(input_tensor)
        det_boxes, det_labels, det_scores = model_beauty.detect_objects(predicted_locs, predicted_scores, min_score=0.8,
                                                                        max_overlap=0.2, top_k=200)

        # Decode class integer labels
        det_labels = [rev_label_map_beauty[l] for l in det_labels[0].to('cpu').tolist()]
        det_labels_np = np.array(det_labels)

        # If no objects found, the bounding box coordinates would be the extremities of the original image
        if det_labels == ['background']:
            row_dict = {'image_path': image_path, 'x0': 0, 'y0':0,
                        'x1': original_image.size[0],'y1':original_image.size[1]}
            if len(df_bb_beauty_val) == 0:
                df_bb_beauty_val = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_beauty_val = df_bb_beauty_val.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Move detections to the CPU
        det_boxes = det_boxes[0].to('cpu')

        # Transform to original image dimensions and to NumPy array
        original_dims = torch.FloatTensor(
            [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
        det_boxes = det_boxes * original_dims
        det_boxes_np = det_boxes.detach().numpy().round().astype(int) + 1

        # Transform scores to NumPy array
        det_scores_np = det_scores[0].to('cpu').detach().numpy()

        # Prioritize 'beauty' label if it exists
        if 'beauty' in det_labels:
            det_scores_filtered = det_scores_np[det_labels_np == 'beauty']
            det_boxes_filtered = det_boxes_np[det_labels_np == 'beauty']

            det_boxes_best = det_boxes_filtered[det_scores_filtered.argmax()].flatten()

            row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
            if len(df_bb_beauty_val) == 0:
                df_bb_beauty_val = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_beauty_val = df_bb_beauty_val.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Record score
        det_boxes_best = det_boxes_np[det_scores_np.argmax()].flatten()
        row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
        if len(df_bb_beauty_val) == 0:
            df_bb_beauty_val = pd.DataFrame(row_dict,index=[0])
        else:
            df_bb_beauty_val = df_bb_beauty_val.append(row_dict, ignore_index=True)

        pbar.update()
        
df_bb_beauty_val.to_csv('data/derived/bb_beauty_val.csv', index=False)

100%|██████████| 76545/76545 [2:33:35<00:00,  8.20it/s]  


Fashion

In [10]:
# Load model checkpoint
checkpoint = 'models/BEST_checkpoint_fashion.pth.tar'
checkpoint = torch.load(checkpoint)
start_epoch = checkpoint['epoch'] + 1
best_loss = checkpoint['best_loss']
print('\nLoaded checkpoint from epoch %d. Best loss so far is %.3f.\n' % (start_epoch, best_loss))
model_fashion = checkpoint['model']
model_fashion = model_fashion.to(device)
model_fashion.eval()


Loaded checkpoint from epoch 95. Best loss so far is 1.527.



SSD300(
  (base): VGGBase(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), 

In [ ]:
df_bb_fashion_train = pd.DataFrame()

with tqdm(total=len(df_fashion_train)) as pbar:
    for image_path in df_fashion_train['image_path']:
        image_path_full = os.path.join('data/raw', image_path)
        original_image = pil_loader(image_path_full)
        input_tensor = transform_pipe(original_image).unsqueeze(0).to(device)
        predicted_locs, predicted_scores = model_fashion(input_tensor)
        det_boxes, det_labels, det_scores = model_fashion.detect_objects(predicted_locs, predicted_scores, min_score=0.8,
                                                                         max_overlap=0.2, top_k=200)

        # Decode class integer labels
        det_labels = [rev_label_map_fashion[l] for l in det_labels[0].to('cpu').tolist()]
        det_labels_np = np.array(det_labels)

        # If no objects found, the bounding box coordinates would be the extremities of the original image
        if det_labels == ['background']:
            row_dict = {'image_path': image_path, 'x0': 0, 'y0':0,
                        'x1': original_image.size[0],'y1':original_image.size[1]}
            if len(df_bb_fashion_train) == 0:
                df_bb_fashion_train = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_fashion_train = df_bb_fashion_train.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Move detections to the CPU
        det_boxes = det_boxes[0].to('cpu')

        # Transform to original image dimensions and to NumPy array
        original_dims = torch.FloatTensor(
            [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
        det_boxes = det_boxes * original_dims
        det_boxes_np = det_boxes.detach().numpy().round().astype(int) + 1

        # Transform scores to NumPy array
        det_scores_np = det_scores[0].to('cpu').detach().numpy()

        # Prioritize 'fashion' label if it exists
        if 'fashion' in det_labels:
            det_scores_filtered = det_scores_np[det_labels_np == 'fashion']
            det_boxes_filtered = det_boxes_np[det_labels_np == 'fashion']

            det_boxes_best = det_boxes_filtered[det_scores_filtered.argmax()].flatten()

            row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
            if len(df_bb_fashion_train) == 0:
                df_bb_fashion_train = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_fashion_train = df_bb_fashion_train.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Record score
        det_boxes_best = det_boxes_np[det_scores_np.argmax()].flatten()
        row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
        if len(df_bb_fashion_train) == 0:
            df_bb_fashion_train = pd.DataFrame(row_dict,index=[0])
        else:
            df_bb_fashion_train = df_bb_fashion_train.append(row_dict, ignore_index=True)

        pbar.update()
        
df_bb_fashion_train.to_csv('data/derived/bb_fashion_train.csv', index=False)

 73%|███████▎  | 201342/275142 [7:54:47<2:32:51,  8.05it/s] 

In [ ]:
df_bb_fashion_val = pd.DataFrame()

with tqdm(total=len(df_fashion_val)) as pbar:
    for image_path in df_fashion_val['image_path']:
        image_path_full = os.path.join('data/raw', image_path)
        original_image = pil_loader(image_path_full)
        input_tensor = transform_pipe(original_image).unsqueeze(0).to(device)
        predicted_locs, predicted_scores = model_fashion(input_tensor)
        det_boxes, det_labels, det_scores = model_fashion.detect_objects(predicted_locs, predicted_scores, min_score=0.8,
                                                                         max_overlap=0.2, top_k=200)

        # Decode class integer labels
        det_labels = [rev_label_map_fashion[l] for l in det_labels[0].to('cpu').tolist()]
        det_labels_np = np.array(det_labels)

        # If no objects found, the bounding box coordinates would be the extremities of the original image
        if det_labels == ['background']:
            row_dict = {'image_path': image_path, 'x0': 0, 'y0':0,
                        'x1': original_image.size[0],'y1':original_image.size[1]}
            if len(df_bb_fashion_val) == 0:
                df_bb_fashion_val = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_fashion_val = df_bb_fashion_val.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Move detections to the CPU
        det_boxes = det_boxes[0].to('cpu')

        # Transform to original image dimensions and to NumPy array
        original_dims = torch.FloatTensor(
            [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
        det_boxes = det_boxes * original_dims
        det_boxes_np = det_boxes.detach().numpy().round().astype(int) + 1

        # Transform scores to NumPy array
        det_scores_np = det_scores[0].to('cpu').detach().numpy()

        # Prioritize 'fashion' label if it exists
        if 'fashion' in det_labels:
            det_scores_filtered = det_scores_np[det_labels_np == 'fashion']
            det_boxes_filtered = det_boxes_np[det_labels_np == 'fashion']

            det_boxes_best = det_boxes_filtered[det_scores_filtered.argmax()].flatten()

            row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
            if len(df_bb_fashion_val) == 0:
                df_bb_fashion_val = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_fashion_val = df_bb_fashion_val.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Record score
        det_boxes_best = det_boxes_np[det_scores_np.argmax()].flatten()
        row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
        if len(df_bb_fashion_val) == 0:
            df_bb_fashion_val = pd.DataFrame(row_dict,index=[0])
        else:
            df_bb_fashion_val = df_bb_fashion_val.append(row_dict, ignore_index=True)

        pbar.update()
        
df_bb_fashion_val.to_csv('data/derived/bb_fashion_val.csv', index=False)

Mobile

In [ ]:
# Load model checkpoint
checkpoint = 'models/BEST_checkpoint_mobile.pth.tar'
checkpoint = torch.load(checkpoint)
start_epoch = checkpoint['epoch'] + 1
best_loss = checkpoint['best_loss']
print('\nLoaded checkpoint from epoch %d. Best loss so far is %.3f.\n' % (start_epoch, best_loss))
model_mobile = checkpoint['model']
model_mobile = model_mobile.to(device)
model_mobile.eval()

In [ ]:
df_bb_mobile_train = pd.DataFrame()

with tqdm(total=len(df_mobile_train)) as pbar:
    for image_path in df_mobile_train['image_path']:
        image_path_full = os.path.join('data/raw', image_path)
        original_image = pil_loader(image_path_full)
        input_tensor = transform_pipe(original_image).unsqueeze(0).to(device)
        predicted_locs, predicted_scores = model_mobile(input_tensor)
        det_boxes, det_labels, det_scores = model_mobile.detect_objects(predicted_locs, predicted_scores, min_score=0.8,
                                                                        max_overlap=0.2, top_k=200)

        # Decode class integer labels
        det_labels = [rev_label_map_mobile[l] for l in det_labels[0].to('cpu').tolist()]
        det_labels_np = np.array(det_labels)

        # If no objects found, the bounding box coordinates would be the extremities of the original image
        if det_labels == ['background']:
            row_dict = {'image_path': image_path, 'x0': 0, 'y0':0,
                        'x1': original_image.size[0],'y1':original_image.size[1]}
            if len(df_bb_mobile_train) == 0:
                df_bb_mobile_train = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_mobile_train = df_bb_mobile_train.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Move detections to the CPU
        det_boxes = det_boxes[0].to('cpu')

        # Transform to original image dimensions and to NumPy array
        original_dims = torch.FloatTensor(
            [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
        det_boxes = det_boxes * original_dims
        det_boxes_np = det_boxes.detach().numpy().round().astype(int) + 1

        # Transform scores to NumPy array
        det_scores_np = det_scores[0].to('cpu').detach().numpy()

        # Prioritize 'mobile' label if it exists
        if 'mobile' in det_labels:
            det_scores_filtered = det_scores_np[det_labels_np == 'mobile']
            det_boxes_filtered = det_boxes_np[det_labels_np == 'mobile']

            det_boxes_best = det_boxes_filtered[det_scores_filtered.argmax()].flatten()

            row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
            if len(df_bb_mobile_train) == 0:
                df_bb_mobile_train = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_mobile_train = df_bb_mobile_train.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Record score
        det_boxes_best = det_boxes_np[det_scores_np.argmax()].flatten()
        row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
        if len(df_bb_mobile_train) == 0:
            df_bb_mobile_train = pd.DataFrame(row_dict,index=[0])
        else:
            df_bb_mobile_train = df_bb_mobile_train.append(row_dict, ignore_index=True)

        pbar.update()
        
df_bb_mobile_train.to_csv('data/derived/bb_mobile_train.csv', index=False)

In [ ]:
df_bb_mobile_val = pd.DataFrame()

with tqdm(total=len(df_mobile_val)) as pbar:
    for image_path in df_mobile_val['image_path']:
        image_path_full = os.path.join('data/raw', image_path)
        original_image = pil_loader(image_path_full)
        input_tensor = transform_pipe(original_image).unsqueeze(0).to(device)
        predicted_locs, predicted_scores = model_mobile(input_tensor)
        det_boxes, det_labels, det_scores = model_mobile.detect_objects(predicted_locs, predicted_scores, min_score=0.8,
                                                                        max_overlap=0.2, top_k=200)

        # Decode class integer labels
        det_labels = [rev_label_map_mobile[l] for l in det_labels[0].to('cpu').tolist()]
        det_labels_np = np.array(det_labels)

        # If no objects found, the bounding box coordinates would be the extremities of the original image
        if det_labels == ['background']:
            row_dict = {'image_path': image_path, 'x0': 0, 'y0':0,
                        'x1': original_image.size[0],'y1':original_image.size[1]}
            if len(df_bb_mobile_val) == 0:
                df_bb_mobile_val = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_mobile_val = df_bb_mobile_val.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Move detections to the CPU
        det_boxes = det_boxes[0].to('cpu')

        # Transform to original image dimensions and to NumPy array
        original_dims = torch.FloatTensor(
            [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
        det_boxes = det_boxes * original_dims
        det_boxes_np = det_boxes.detach().numpy().round().astype(int) + 1

        # Transform scores to NumPy array
        det_scores_np = det_scores[0].to('cpu').detach().numpy()

        # Prioritize 'mobile' label if it exists
        if 'mobile' in det_labels:
            det_scores_filtered = det_scores_np[det_labels_np == 'mobile']
            det_boxes_filtered = det_boxes_np[det_labels_np == 'mobile']

            det_boxes_best = det_boxes_filtered[det_scores_filtered.argmax()].flatten()

            row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
            if len(df_bb_mobile_val) == 0:
                df_bb_mobile_val = pd.DataFrame(row_dict,index=[0])
            else:
                df_bb_mobile_val = df_bb_mobile_val.append(row_dict, ignore_index=True)

            pbar.update()
            continue

        # Record score
        det_boxes_best = det_boxes_np[det_scores_np.argmax()].flatten()
        row_dict = {'image_path': image_path, 'x0': det_boxes_best[0], 'y0':det_boxes_best[1],
                        'x1': det_boxes_best[2],'y1':det_boxes_best[3]}
        if len(df_bb_mobile_val) == 0:
            df_bb_mobile_val = pd.DataFrame(row_dict,index=[0])
        else:
            df_bb_mobile_val = df_bb_mobile_val.append(row_dict, ignore_index=True)

        pbar.update()
        
df_bb_mobile_val.to_csv('data/derived/bb_mobile_val.csv', index=False)